# Instant Compaction with Session Memory

Traditional compaction is slow: when you hit the context limit, you wait for a summary.

With **Instant compaction** the session memory is proactively generated once a soft token threshold is reached. Once the user triggers a compaction or a hard limit is reached, the summary is already available, so the user doesn't need to wait.

Result: Instant compaction, no waiting.


```
TRADITIONAL COMPACTION (slow)
─────────────────────────────
Turn 1 → Turn 2 → Turn 3 → ... → Turn N → CONTEXT FULL!
                                              │
                                              ▼
                                    ┌─────────────────┐
                                    │ Generate summary│
                                    │ ( USER WAITS !) │
                                    └─────────────────┘
                                              │
                                              ▼
                                         Continue


SESSION MEMORY COMPACTION (instant)
────────────────────────────────────
Turn 1 → Turn 2 → ... → Turn K → Turn K+1 → ... → Turn N → ..  → CONTEXT FULL!
                            │                         │            │
                (soft threshold met:              (update          │
                   10k tokens init)                trigger)        │
                            │                                      │
                            │                         │            │
                            ▼                         ▼            │
                       ┌────────┐                ┌────────┐        │
                       │ Update │                │ Update │        │
                       │ memory │ (background)   │ memory │        │
                       └────────┘                └────────┘        │
                            │                         │            │
                            ▼                         ▼            ▼
                     📝 session-memory.md ──────────────────► INSTANT SWAP!
                       (continuously updated)
```

**Update triggers:** The first summary is generated after the initial 10k tokens. Updates can be triggered after every subsequent turn, or at periodically at natural breakpoints intervals (e.g. every ~5k tokens or 3+ tool calls).

## Fundamentals: writing a compaction prompt

Make sure you have a well structured session memory prompt. 

Some best practices include:
- Use chain-of-thought before summarizing — analyze first, then output                                                                                         
- Enumerate exactly what to preserve: file paths, code snippets, errors, user corrections                                                                      
- Weight recency heavily — the end of the conversation is the active context                                                                                   
- Require verbatim quotes for next steps to prevent task drift                                                                                                 
- Use structured sections with token budgets per section                                                                                                       
- Include a "Current State" section that always reflects the moment of compaction

Some pitfalls include:
- Vague prompts like "summarize this conversation" produce lossy output                                                                                        
- Treating all messages equally loses the active working context                                                                                               
- Paraphrasing next steps introduces subtle drift that compounds                                                                                               
- Omitting error history causes the model to retry failed approaches                                                                                           
- Dropping user corrections makes the model revert to old behaviors                                                                                            
- No token limits lets one section consume the entire summary                                                                                                  
- Summarizing for human readability instead of model continuity
- Having the agent try to compress the results of tool calls here - this can be retrieved later if the agent needs it


In [180]:
SESSION_CREATION_PROMPT = """
<analysis-instructions>
Before generating your summary, analyze the transcript in <think>...</think> tags:
1. What did the user originally request? (Exact phrasing)
2. What actions succeeded? What failed and why?
3. Did the user correct or redirect the assistant at any point?
4. What was actively being worked on at the end?
5. What tasks remain incomplete or pending?
6. What specific details (IDs, paths, values, names) must survive compression?
</analysis-instructions>

<summary-format>
## User Intent
The user's original request and any refinements. Use direct quotes for key requirements.
If the user's goal evolved during the conversation, capture that progression.

## Completed Work
Actions successfully performed. Be specific:
- What was created, modified, or deleted
- Exact identifiers (file paths, record IDs, URLs, names)
- Specific values, configurations, or settings applied

## Errors & Corrections
- Problems encountered and how they were resolved
- Approaches that failed (so they aren't retried)
- User corrections: "don't do X", "actually I meant Y", "that's wrong because..."
Capture corrections verbatim—these represent learned preferences.

## Active Work
What was in progress when the session ended. Include:
- The specific task being performed
- Direct quotes showing exactly where work left off
- Any partial results or intermediate state

## Pending Tasks
Remaining items the user requested that haven't been started.
Distinguish between "explicitly requested" and "implied/assumed."

## Key References
Important details needed to continue:
- Identifiers: IDs, paths, URLs, names, keys
- Values: numbers, dates, configurations, credentials (redacted)
- Context: relevant background information, constraints, preferences
- Citations: sources referenced during the conversation
</summary-format>

<preserve-rules>
Always preserve when present:
- Exact identifiers (IDs, paths, URLs, keys, names)
- Error messages verbatim
- User corrections and negative feedback
- Specific values, formulas, or configurations
- Technical constraints or requirements discovered
- The precise state of any in-progress work
</preserve-rules>

<compression-rules>
- Weight recent messages more heavily—the end of the transcript is the active context
- Omit pleasantries, acknowledgments, and filler ("Sure!", "Great question")
- Omit system context that will be re-injected separately
- Keep each section under 500 words; condense older content to make room for recent
- If you must cut details, preserve: user corrections > errors > active work > completed work
</compression-rules>
"""

## Traditional compacting example
In traditional compaction, you generate one summary once the token threshold is reached.

In [115]:
# setup, we are using haiku for demo purposes
import anthropic
import warnings

# Suppress noisy FutureWarning from coconut compiler
warnings.filterwarnings("ignore", category=FutureWarning, module="coconut")

client = anthropic.Anthropic()
MODEL = "claude-haiku-4-5-20251001"

# helper functions:

def truncate_response(text: str, max_lines: int = 8) -> str:
    """Truncate long responses for cleaner output display."""
    lines = text.strip().split("\n")
    if len(lines) <= max_lines:
        return text
    return "\n".join(lines[:max_lines]) + f"\n... ({len(lines) - max_lines} more lines)"

def build_transcript(messages: list[dict]) -> str:
    lines = []
    for msg in messages:
        role = "User" if msg["role"] == "user" else "Assistant"
        lines.append(f"{role}: {msg['content']}")
    return "\n\n".join(lines)

def remove_thinking_blocks(text: str):                                                                                       
    """Remove <think>...</think> blocks from the text."""                                                                    
    import re                                                                                                                
    matches = re.findall(r"<think>.*?</think>", text, flags=re.DOTALL)                                                       
    cleaned = re.sub(r"<think>.*?</think>\s*", "", text, flags=re.DOTALL).strip()                                            
    return cleaned, "".join(matches)         

In [ ]:
import time

class TraditionalCompactingChatSession:
    """Traditional chat session with compaction after the fact."""
    def __init__(self, context_limit: int = 1500):
        self.context_limit = context_limit # the point at which the conversation is compacted so it does not exceed model limits. You would set this based on your model's context window size with a buffer for response tokens.
        self.messages = []
        self.current_context_window_tokens = 0
        self.summary = None
    
    def chat(self, user_message: str):
        # In traditional compaction, we check if we need to compact when the user sends a message. NOT IDEAL!
        if self.current_context_window_tokens >= self.context_limit:
            print(f"\n🧹 Context window at {self.current_context_window_tokens} tokens. Limit exceeded, compacting session memory...")
            self.compact() # compacts everything before the new user message
        
        self.messages.append({"role": "user", "content": user_message})
        
        response = client.messages.create(
            model=MODEL,
            max_tokens=1024,
            system="You are a helpful coding assistant.",
            messages=self.messages
        )
        
        assistant_message = response.content[0].text
        self.messages.append({"role": "assistant", "content": assistant_message})
        
        # approximate current token count in the conversation before the next user message
        self.current_context_window_tokens = response.usage.input_tokens + response.usage.output_tokens

        return assistant_message, response.usage
    
    def compact(self):
        prev_msg_count = len(self.messages)
        
        compaction_prompt = SESSION_CREATION_PROMPT + "\n\nTranscript:\n" + build_transcript(self.messages)
        
        start_time = time.perf_counter()
        response = client.messages.create(
            model=MODEL,
            max_tokens=800, # note that some of this will be eaten up by the thinking blocks we remove later
            system="""You are a session memory agent. Compress the conversation into a structured summary 
that preserves all information needed to continue work seamlessly. Optimize for the assistant's 
ability to continue working, not human readability""",
            messages=[{"role": "user", "content": compaction_prompt}]
        )
        elapsed = time.perf_counter() - start_time
        
        # Generate new summary message
        self.summary, removed_text = remove_thinking_blocks(response.content[0].text) # clean up any <think> blocks because they are not needed in the session memory
        approximate_summary_tokens = response.usage.output_tokens - round(len(removed_text) / 4)  # rough estimate of tokens removed from summary
       
        # Replace prior messages with new summary message
        self.messages = [{
            "role": "user",
            "content": f"""This session is being continued from a previous conversation. Here is the session memory: {self.summary}.Continue from where we left off."""
        }]

        # Show stats on compaction
        curr_msg_count = len(self.messages)
        
        # Show token reduction if we just compacted
        reduction = self.current_context_window_tokens - approximate_summary_tokens
        pct = (reduction / self.current_context_window_tokens) * 100
        
        print(f"\n{'-' * 60}")
        print(f"✅ Tokens reduced: {self.current_context_window_tokens:,} → {approximate_summary_tokens:.0f} ({reduction:,} tokens saved, {pct:.0f}% reduction)")
        print(f"📝 New session memory created.")
        print(f"🔄 Compaction messages: {prev_msg_count} → {curr_msg_count}")
        print(f"⏱️ Compaction time: {elapsed:.2f}s (user waiting...)")
        print(f"{'-' * 60}")
        
        # Update token count to reflect compacted state
        self.current_context_window_tokens = approximate_summary_tokens

### Example use of traditional compaction

In [138]:
session = TraditionalCompactingChatSession()

messages = [
    "Explain Python decorators with a simple example.",
    "Now show me a decorator that logs function arguments.",
    "How do I make a decorator that accepts parameters?",
]

print("Starting conversation with traditional compacting chat session...\n")

turn_count = 0

for i, message in enumerate(messages, 1):
    response, usage = session.chat(message)
    turn_count += 1
    print(
        f"==============================================\n"
        f"Turn {turn_count}: Input={usage.input_tokens:,} | "
        f"Output={usage.output_tokens:,} | "
        f"Messages={len(session.messages)}"
    )
    print(f"\nUser: {message}")
    print(f"\nAssistant: \n{truncate_response(response, max_lines=3)}")
    print()

Starting conversation with traditional compacting chat session...

Turn 1: Input=41 | Output=450 | Messages=2

User: Explain Python decorators with a simple example.

Assistant: 
# Python Decorators Explained

A **decorator** is a function that modifies or enhances another function or class without permanently changing its source code.
... (69 more lines)

Turn 2: Input=504 | Output=871 | Messages=4

User: Now show me a decorator that logs function arguments.

Assistant: 
# Decorator that Logs Function Arguments

Here's a practical logging decorator:
... (122 more lines)

Turn 3: Input=1,388 | Output=1,024 | Messages=6

User: How do I make a decorator that accepts parameters?

Assistant: 
# Decorators with Parameters

To make a decorator that accepts parameters, you need an **extra layer of nesting**.
... (144 more lines)



In [139]:
response, _ = session.chat("What other related topics should we cover?")
print("Final response after compaction:")
print(response)


🧹 Context window at 2412 tokens. Limit exceeded, compacting session memory...

------------------------------------------------------------
✅ Tokens reduced: 2,412 → 391 (2,021 tokens saved, 84% reduction)
📝 New session memory created.
🔄 Compaction messages: 6 → 1
⏱️ Compaction time: 6.93s (user waiting...)
------------------------------------------------------------
Final response after compaction:
# Continuing: Complete the `validate_types` Decorator

Let me finish that incomplete example from Response 3:

```python
from functools import wraps

def validate_types(**type_checks):
    """Decorator that validates argument types before execution"""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Get function signature to match args to parameter names
            import inspect
            sig = inspect.signature(func)
            bound_args = sig.bind(*args, **kwargs)
            bound_args.apply_defaults()
            
            # Val

In [140]:
print(session.summary)

## User Intent
User requested progressive education on Python decorators:
1. "Explain Python decorators with a simple example"
2. "Now show me a decorator that logs function arguments"
3. "How do I make a decorator that accepts parameters?"

User is learning decorator progression from basic to advanced (parameterized).

## Completed Work
- **Response 1**: Basic decorator explanation with gift-wrapping analogy, simple example (`my_decorator`), example with arguments (`*args, **kwargs`), and real-world use cases
- **Response 2**: Logging decorator (`log_arguments`) in three versions:
  - Basic version logging args, kwargs, and result
  - Enhanced version with formatted output and emoji
  - Professional version using `functools.wraps` to preserve function metadata
  - Applied to example functions: `greet()`, `multiply()`, `calculate()`, `divide()`
- **Response 3**: Parameterized decorators with multiple practical examples:
  - `repeat(times)` - repeats function execution N times
  - `rate

As a result the user experineces a wait time when compaction occurs. It is only a few seconds in this example, but for long context compaction, this can be must longer.

## Instant Compaction


The key insight: **build the session memory in the background** so it's ready when you need it.

```
Turn 1 → Turn 2 → ... → Turn K  → Turn K+1 → ... → CONTEXT FULL!
                           │           │                 │
                     (threshold)  (update)          INSTANT!
                           ↓           ↓                 ↓
                    [Background]  [Background]    [Just swap in
                     memory init   memory update   pre-built memory]
```

This `InstantCompactingChatSession` class uses **threading** for background execution:
1. **`threading.Thread`** - runs memory updates in background without blocking
2. **Thread-safe state** - uses `threading.Lock` to safely update shared memory
3. **Daemon threads** - background work doesn't prevent program exit
4. **Instant compaction** - when context is full, just swap in the pre-built memory

In [181]:
import threading
import time


class InstantCompactingChatSession:
    """
    Maintains session memory via incremental background updates.
    
    Key insight: By updating memory in the background after each turn,
    the summary is already ready when compaction is needed - instant swap!
    """

    def __init__(
        self,
        context_limit: int = 1500,
        min_tokens_to_init: int = 700,
        min_tokens_between_updates: int = 300,
    ):
        # Thresholds
        self.context_limit = context_limit # the point at which the conversation is compacted so it does not exceed model limits
        self.min_tokens_to_init = min_tokens_to_init # tokens needed to trigger initial memory creation; note this happens PROACTIVELY in background unlike traditional compaction
        self.min_tokens_between_updates = min_tokens_between_updates # tokens needed to trigger memory update. only comes into play after initial memory is created and additional compaction (memory update) is needed after that

        # Conversation state
        self.messages = []
        self.current_context_window_tokens = 0

        # Session memory state
        self.session_memory = None # this is the compacted conversation in session memory; for the demo we are storing this in memory, but in production you would write to session_memory.md file
        self.last_summarized_index = 0 # The index of the last message included in the session memory
        self.tokens_at_last_update = 0 # TBD if I need this

        # Background update tracking
        self._update_thread: threading.Thread | None = None
        self.last_update_time = None
        self._lock = threading.Lock()

    def chat(self, user_message: str):
        """Process a chat turn with background session memory updates."""
        if self.current_context_window_tokens >= self.context_limit:
            self.compact() # note that when this is triggered, the compaction has already been created and is just swapped in instantly

        self.messages.append({"role": "user", "content": user_message})

        response = client.messages.create(
            model=MODEL,
            max_tokens=1024,
            system="You are a helpful coding assistant.",
            messages=self.messages,
        )

        assistant_message = response.content[0].text
        self.messages.append({"role": "assistant", "content": assistant_message})

        # approximate current token count in the conversation before the next user message
        self.current_context_window_tokens = response.usage.input_tokens + response.usage.output_tokens

        # KEY DIFFERENCE: Trigger background memory update if needed proactively, before compaction is needed
        background_status = None
        if self._should_init_memory() or self._should_update_memory():
            self._trigger_background_update()
            background_status = "initializing" if self.session_memory is None else "updating"

        return assistant_message, response.usage, background_status
    
    # Helper methods to determine when to init/update/compact
    def _should_init_memory(self) -> bool:
        return (
            self.session_memory is None
            and self.current_context_window_tokens >= self.min_tokens_to_init
        )

    # Helper method to determine if memory should be updated
    def _should_update_memory(self) -> bool:
        if self.session_memory is None:
            return False
        tokens_since = self.current_context_window_tokens - self.tokens_at_last_update
        return tokens_since >= self.min_tokens_between_updates

    # Methods to create initial session memory
    def _create_session_memory(self, messages: list[dict]) -> str:
        """Generate initial session memory from messages."""
        transcript = build_transcript(messages)

        response = client.messages.create(
            model=MODEL,
            max_tokens=1024,
            system="""You are a session memory agent. Compress the conversation into a structured summary 
that preserves all information needed to continue work seamlessly. Optimize for the assistant's 
ability to continue working, not human readability.""",
            messages=[
                {
                    "role": "user",
                    "content": f"""Conversation transcript:
{transcript}

Create session memory using these instructions:
{SESSION_CREATION_PROMPT}

First analyze in <think>...</think> tags, then output the structured summary.""",
                }
            ],
        )
        summary, _ = remove_thinking_blocks(response.content[0].text)  # clean up any <think> blocks because they are not needed in the session memory
        return summary

    def _update_session_memory(self, new_messages: list[dict]) -> str:
        """Update existing session memory with new messages. In practice, you may want to do this via file edit rather than full re-generation. But for demo purposes we do full regeneration here."""
        transcript = build_transcript(new_messages)

        response = client.messages.create(
            model=MODEL,
            max_tokens=1024,
            system="""You are a session memory agent. Update the existing session memory with new information 
from the recent conversation. Preserve important existing details while integrating new content.""",
            messages=[
                {
                    "role": "user",
                    "content": f"""Current session memory:
{self.session_memory}

New messages to integrate:
{transcript}

Update the session memory following these guidelines:
{SESSION_CREATION_PROMPT}

Output only the updated session memory (no analysis tags needed for updates).

First analyze in <think>...</think> tags, then output the updated structured summary.""",
                }
            ],
        )
        updated_summary, _ = remove_thinking_blocks(response.content[0].text)  # clean up any <think> blocks because they are not needed in the session memory
        return updated_summary

    # Background memory update methods
    def _background_memory_update(
        self, messages_snapshot: list[dict], snapshot_index: int, current_tokens: int
    ):
        """Run session memory update in a background thread."""
        try:
            with self._lock:
                current_session_memory = self.session_memory
                last_index = self.last_summarized_index

            if current_session_memory is None:
                new_memory = self._create_session_memory(messages_snapshot)
            else:
                # Get new messages since last summary
                new_messages = messages_snapshot[last_index :]
                if not new_messages:
                    return
                new_memory = self._update_session_memory(new_messages)

            # Update state (thread-safe)
            with self._lock:
                self.session_memory = new_memory
                self.last_summarized_index = snapshot_index
                self.tokens_at_last_update = current_tokens
                self.last_update_time = time.time()

        except Exception as e:
            print(f"   [Background] Error updating memory: {e}")

    # This makes sure only one background update runs at a time. If one is already running, we skip starting another. If not, we start a new thread to do the update.
    def _trigger_background_update(self):
        """Trigger a background session memory update."""
        if self._update_thread is not None and self._update_thread.is_alive():
            return

        messages_snapshot = self.messages.copy()
        snapshot_index = len(messages_snapshot)
        current_tokens = self.current_context_window_tokens

        self._update_thread = threading.Thread(
            target=self._background_memory_update,
            args=(messages_snapshot, snapshot_index, current_tokens),
            daemon=True,
        )
        self._update_thread.start()

    # Function to compact
    def compact(self):
        """INSTANT compaction using pre-built session memory."""
        prev_msg_count = len(self.messages)

        # Ensure session memory is ready. Shouldn't be an issue normally, but here for safety.
        if self.session_memory is None:
            if self._update_thread is not None and self._update_thread.is_alive():
                print("   ⏳ Waiting for background memory update...")
                self._update_thread.join(timeout=30.0)

            if self.session_memory is None:
                print("   ⚠️  No pre-built memory, creating synchronously...")
                start = time.perf_counter()
                self.session_memory = self._create_session_memory(self.messages)
                elapsed = time.perf_counter() - start
                print(f"   ⏱️  Took {elapsed:.2f}s (but should be instant normally!)")
                self.last_summarized_index = len(self.messages)

        with self._lock:
            unsummarized = self.messages[self.last_summarized_index :]

            summary_message = {
                "role": "user",
                "content": f"""This session is being continued from a previous conversation.

    Here is the session memory:
    {self.session_memory}

    Continue from where we left off.""",
            }

            self.messages = [summary_message] + unsummarized
            self.last_summarized_index = 1

            print(f"\n{'=' * 60}")
            print(f"⚡ INSTANT COMPACTION! Messages: {prev_msg_count} → {len(self.messages)}")
            print(f"   Session memory was pre-built (no wait time!)")
            print(f"{'=' * 60}")

    

### Example use of Instant Compaction

In [169]:
# Low thresholds for demo - in production you'd use higher values
session = InstantCompactingChatSession(
    context_limit=1500,
    min_tokens_to_init=700,
    min_tokens_between_updates=300,
)

messages = [
    "Explain Python decorators with a simple example.",
    "Now show me a decorator that logs function arguments.",
    "How do I make a decorator that accepts parameters?",
]
print("Starting conversation with instant compacting chat session...\n")

turn_count = 0
for i, message in enumerate(messages, 1):
    response, usage, background_status = session.chat(message)
    turn_count += 1
    print(
        f"==============================================\n"
        f"Turn {turn_count}: "
    )
    print(f"\nUser: {message}")
    print(f"\nAssistant: \n{truncate_response(response, max_lines=3)}")
    print(f"\n \nTurn end state: "
        f"\nInput={usage.input_tokens:,} |"
        f"Output={usage.output_tokens:,} | "
        f"Messages={len(session.messages)} | "
        f"Memory: {'ready' if session.session_memory else 'no memory created yet'}\n"
    )
    if background_status:
        print("On the next response, the current conversation tokens will be:", session.current_context_window_tokens)
        print(f"   [Background] Proactively {background_status} session memory...")
    # Sleep to allow background updates to complete for demo purposes
    if i < len(messages):
        time.sleep(5)

Starting conversation with instant compacting chat session...

Turn 1: 

User: Explain Python decorators with a simple example.

Assistant: 
# Python Decorators Explained

A **decorator** is a function that modifies or enhances another function or class without permanently changing its source code. It wraps a function with additional functionality.
... (66 more lines)

 
Turn end state: 
Input=41 |Output=418 | Messages=2 | Memory: no memory created yet

Turn 2: 

User: Now show me a decorator that logs function arguments.

Assistant: 
# Function Argument Logging Decorator

Here are several approaches, from simple to advanced:
... (104 more lines)

 
Turn end state: 
Input=472 |Output=750 | Messages=4 | Memory: no memory created yet

On the next response, the current conversation tokens will be: 1222
   [Background] Proactively initializing session memory...
Turn 3: 

User: How do I make a decorator that accepts parameters?

Assistant: 
# Decorators with Parameters

To create a decorato

In [170]:
message = "What did we just talk about?"
response, usage, background_status = session.chat(message)
print(f"\nUser: {message}")
print(f"\nAssistant: \n{truncate_response(response, max_lines=3)}")
print(f"\n \nTurn end state: "
    f"\nInput={usage.input_tokens:,} |"
    f"Output={usage.output_tokens:,} | "
    f"Messages={len(session.messages)} | "
    f"Memory: {'ready' if session.session_memory else 'no memory created yet'}\n"
)


⚡ INSTANT COMPACTION! Messages: 6 → 1
   Kept 0 unsummarized messages
   Session memory was pre-built (no wait time!)

User: What did we just talk about?

Assistant: 
# Session Summary

We just completed a comprehensive discussion on **parameterized decorators** (your third request in this session).
... (33 more lines)

 
Turn end state: 
Input=778 |Output=386 | Messages=3 | Memory: ready



In [171]:
message = "What are some good follow up topics we should cover?"
response, usage, background_status = session.chat(message)
print(f"\nUser: {message}")
print(f"\nAssistant: \n{truncate_response(response, max_lines=3)}")
print(f"\n \nTurn end state: "
    f"\nInput={usage.input_tokens:,} |"
    f"Output={usage.output_tokens:,} | "
    f"Messages={len(session.messages)} | "
    f"Memory: {'ready' if session.session_memory else 'no memory created yet'}\n"
)


User: What are some good follow up topics we should cover?

Assistant: 
# Recommended Follow-Up Topics

Based on your progression through decorator fundamentals → logging → parameterized decorators, here are the most logical next steps:
... (63 more lines)

 
Turn end state: 
Input=1,178 |Output=723 | Messages=5 | Memory: ready



## Advanced: Adding Prompt Caching


The background updates can be made **~10x cheaper** by using prompt caching. The trick:
1. Pass the **full conversation** to the background summarizer
2. Add `cache_control` markers so subsequent requests hit the cache
3. Only the new "summarize this" instruction is billed at full price

```
┌─────────────────────────────────────────────────────────────────────────────────┐
│                    PROMPT CACHING FOR LONG CONVERSATIONS                        │
├─────────────────────────────────────────────────────────────────────────────────┤
│                                                                                 │
│  WITHOUT CACHING: Pay full price for entire context every turn                 │
│  ════════════════════════════════════════════════════════════                   │
│                                                                                 │
│  Turn 1:  [System][User1][Asst1]                         →  500 tokens  @ $3/M │
│  Turn 2:  [System][User1][Asst1][User2][Asst2]           → 1500 tokens  @ $3/M │
│  Turn 3:  [System][User1][Asst1][User2][Asst2][User3]... → 3000 tokens  @ $3/M │
│  Turn 4:  [System][User1][Asst1][User2][Asst2][User3]... → 5000 tokens  @ $3/M │
│           ─────────────────────────────────────────────                         │
│                                              Total: 10,000 tokens = $0.030      │
│                                                                                 │
│                                                                                 │
│  WITH CACHING: Pay full price once, then 90% discount on prefix                │
│  ═══════════════════════════════════════════════════════════════                │
│                                                                                 │
│  Turn 1:  [System][User1][Asst1]◆                        →  500 tokens  @ $3/M │
│                                ▲                            (cache created)    │
│                          cache breakpoint                                       │
│                                                                                 │
│  Turn 2:  [System][User1][Asst1][User2][Asst2]◆                                │
│           ╰─────── cached ──────╯                                              │
│                500 @ $0.30/M + 1000 new @ $3/M  =  $0.0032                     │
│                                                                                 │
│  Turn 3:  [System][User1][Asst1][User2][Asst2][User3][Asst3]◆                  │
│           ╰──────────── cached ─────────────╯                                  │
│               1500 @ $0.30/M + 1500 new @ $3/M  =  $0.0050                     │
│                                                                                 │
│  Turn 4:  [System][User1][Asst1][User2][Asst2][User3][Asst3][User4][Asst4]◆    │
│           ╰───────────────────── cached ─────────────────────╯                 │
│                     3000 @ $0.30/M + 2000 new @ $3/M  =  $0.0069               │
│           ─────────────────────────────────────────────                         │
│                                              Total: $0.0166  (45% savings)     │
│                                                                                 │
├─────────────────────────────────────────────────────────────────────────────────┤
│                                                                                 │
│  COMPACTION + CACHING: Double benefit                                           │
│  ════════════════════════════════════                                           │
│                                                                                 │
│    Main Chat                      Background Summarizer                         │
│    ─────────                      ─────────────────────                         │
│                                                                                 │
│  [Conversation grows...]          [Same conversation prefix]◆ + [Summarize!]   │
│         │                                    │                                  │
│         │                         Cache hit! Only pays for                      │
│         │                         the summarization prompt                      │
│         │                                    │                                  │
│         ▼                                    ▼                                  │
│  Context limit reached  ──────►  Session memory ready instantly                │
│                                  (built cheaply in background)                  │
│                                                                                 │
│  ┌──────────────────────────────────────────────────────────────────────────┐  │
│  │  Key insight: The background summarizer reuses the same conversation     │  │
│  │  prefix that was just sent to the main chat - automatic cache hit!       │  │
│  └──────────────────────────────────────────────────────────────────────────┘  │
│                                                                                 │
└─────────────────────────────────────────────────────────────────────────────────┘

◆ = cache_control breakpoint (cache everything before this point)
```

### Why this matters for compaction

| Scenario | Cost per background update | Notes |
|----------|---------------------------|-------|
| No caching | Full input cost | 5,000 tokens × $3/M = $0.015 |
| With caching | ~10% of input cost | 500 new + 4,500 cached = $0.003 |
| **Savings** | **~80%** | Compounds over many updates |

The longer the conversation, the bigger the savings—exactly when you need compaction most!

### How the Caching Works

The key is in `_add_cache_control()` and `_create_session_memory_cached()`:

```python
# 1. Mark the last conversation message with cache_control
{
    "role": "user",
    "content": [{
        "type": "text",
        "text": msg["content"],
        "cache_control": {"type": "ephemeral"}  # <-- This creates a cache breakpoint
    }]
}

# 2. Also mark the system prompt
system=[{
    "type": "text",
    "text": "You are a session memory agent...",
    "cache_control": {"type": "ephemeral"}
}]
```

**Why this works:**
- The first background update creates a cache entry for `[System + Messages]`
- Subsequent updates with the same message prefix get **cache hits**
- Only the new summarization instruction is billed at full price
- Cache entries have a 5-minute TTL, so rapid updates benefit most

**Cost math:**
- Without caching: 5,000 tokens × $3.00/1M = $0.015 per update
- With caching: 500 new tokens × $3.00/1M + 4,500 cached × $0.30/1M = $0.00285
- **Savings: ~80%** on background summarization costs

In [182]:
SMARTER_MODEL = "claude-sonnet-4-5-20250929"
class CachedInstantCompactingChatSession(InstantCompactingChatSession):
    """Instant compacting session with prompt caching enabled."""
    
    def _add_cache_control(self, messages: list[dict]) -> list[dict]:
        """Convert all messages to list format for consistent structure, with cache_control on the last message.

        For prompt caching to work, the message prefix structure must be identical between requests.
        If we only convert the last message to list format, previous messages change from list→string
        on the next turn, breaking the cache match.
        """
        if not messages:
            return messages

        cached_messages = []
        for i, msg in enumerate(messages):
            is_last = (i == len(messages) - 1)
            content_block = {
                "type": "text",
                "text": msg["content"],
            }
            if is_last:
                content_block["cache_control"] = {"type": "ephemeral"}

            cached_messages.append({
                "role": msg["role"],
                "content": [content_block],
            })

        return cached_messages

    def chat(self, user_message: str):
        if self.current_context_window_tokens >= self.context_limit:
            self.compact()

        self.messages.append({"role": "user", "content": user_message})

        response = client.messages.create(
            model=SMARTER_MODEL,
            max_tokens=1024,
            system="You are a helpful coding assistant.",
            messages=self._add_cache_control(self.messages),
        )

        assistant_message = response.content[0].text
        self.messages.append({"role": "assistant", "content": assistant_message})

        self.current_context_window_tokens = response.usage.input_tokens + response.usage.output_tokens

        background_status = None
        if self._should_init_memory() or self._should_update_memory():
            self._trigger_background_update()
            background_status = "initializing" if self.session_memory is None else "updating"

        return assistant_message, response.usage, background_status

    def _create_session_memory(self, messages: list[dict]) -> str:
        transcript = build_transcript(messages)

        prompt_messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"""Conversation transcript:
{transcript}

Create session memory using these instructions:
{SESSION_CREATION_PROMPT}

First analyze in <think>...</think> tags, then output the structured summary.""",
                        "cache_control": {"type": "ephemeral"},
                    }
                ],
            }
        ]

        response = client.messages.create(
            model=SMARTER_MODEL,
            max_tokens=1024,
            system="""You are a session memory agent. Compress the conversation into a structured summary 
that preserves all information needed to continue work seamlessly. Optimize for the assistant's 
ability to continue working, not human readability.""",
            messages=prompt_messages,
        )
        summary, _ = remove_thinking_blocks(response.content[0].text)
        return summary

    def _update_session_memory(self, new_messages: list[dict]) -> str:
        transcript = build_transcript(new_messages)

        prompt_messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": f"""Current session memory:
{self.session_memory}

New messages to integrate:
{transcript}

Update the session memory following these guidelines:
{SESSION_CREATION_PROMPT}

Output only the updated session memory (no analysis tags needed for updates).

First analyze in <think>...</think> tags, then output the updated structured summary.""",
                        "cache_control": {"type": "ephemeral"},
                    }
                ],
            }
        ]

        response = client.messages.create(
            model=SMARTER_MODEL,
            max_tokens=1024,
            system="""You are a session memory agent. Update the existing session memory with new information 
from the recent conversation. Preserve important existing details while integrating new content.""",
            messages=prompt_messages,
        )
        updated_summary, _ = remove_thinking_blocks(response.content[0].text)
        return updated_summary

In [183]:
# Low thresholds for demo - in production you'd use higher values
session = CachedInstantCompactingChatSession(
    context_limit=2500,
    min_tokens_to_init=1000,
    min_tokens_between_updates=500,
)

messages = [
    "Explain Python decorators with a simple example.",
    "Now show me a decorator that logs function arguments.",
    "How do I make a decorator that accepts parameters?",
]
print("Starting conversation with CACHED instant compacting chat session...\n")

turn_count = 0
for i, message in enumerate(messages, 1):
    response, usage, bg_status = session.chat(message)
    turn_count += 1
    
    # Cache stats
    cache_created = getattr(usage, 'cache_creation_input_tokens', 0) or 0
    cache_read = getattr(usage, 'cache_read_input_tokens', 0) or 0
    cache_hit_pct = (cache_read / usage.input_tokens * 100) if usage.input_tokens > 0 else 0
    
    print(
        f"==============================================\n"
        f"Turn {turn_count}:"
    )
    print(f"\nUser: {message}")
    print(f"\nAssistant: \n{truncate_response(response, max_lines=3)}")
    print(
        f"\nTurn end state:"
        f"\n  Input={usage.input_tokens:,} | Output={usage.output_tokens:,}"
        f"\n  Cache: {cache_read:,} read, {cache_created:,} created ({cache_hit_pct:.0f}% hit rate)"
        f"\n  Messages={len(session.messages)} | Memory: {'ready' if session.session_memory else 'not yet'}"
    )
    
    if bg_status:
        print(f"\n  [Background] Proactively {bg_status} session memory...")
    
    if i < len(messages):
        time.sleep(5)

Starting conversation with CACHED instant compacting chat session...

Turn 1:

User: Explain Python decorators with a simple example.

Assistant: 
# Python Decorators Explained

## What is a Decorator?
... (66 more lines)

Turn end state:
  Input=41 | Output=422
  Cache: 0 read, 0 created (0% hit rate)
  Messages=2 | Memory: not yet
Turn 2:

User: Now show me a decorator that logs function arguments.

Assistant: 
# Decorator that Logs Function Arguments

## Basic Logging Decorator
... (124 more lines)

Turn end state:
  Input=476 | Output=1,024
  Cache: 0 read, 0 created (0% hit rate)
  Messages=4 | Memory: not yet

  [Background] Proactively initializing session memory...
Turn 3:

User: How do I make a decorator that accepts parameters?

Assistant: 
# Decorators with Parameters

## The Pattern: Three Levels of Functions
... (144 more lines)

Turn end state:
  Input=1,516 | Output=1,024
  Cache: 0 read, 0 created (0% hit rate)
  Messages=6 | Memory: not yet

  [Background] Proactively 

In [184]:
message = "What did we just talk about?"
response, usage, background_status = session.chat(message)
print(f"\nUser: {message}")
print(f"\nAssistant: \n{truncate_response(response, max_lines=3)}")
print(f"\n \nTurn end state: "
    f"\nInput={usage.input_tokens:,} |"
    f"Output={usage.output_tokens:,} | "
    f"Messages={len(session.messages)} | "
    f"Memory: {'ready' if session.session_memory else 'no memory created yet'}\n"
)


⚡ INSTANT COMPACTION! Messages: 6 → 3
   Session memory was pre-built (no wait time!)

User: What did we just talk about?

Assistant: 
We just talked about **decorators with parameters** in Python!

## Quick Summary:
... (28 more lines)

 
Turn end state: 
Input=1,780 |Output=347 | Messages=5 | Memory: ready



In [ ]:
# Debug: Print the structure of _add_cache_control output
import json

cached_messages = session._add_cache_control(session.messages)
print("Number of messages:", len(cached_messages))
print("\nStructure of cached messages:")
for i, msg in enumerate(cached_messages):
    print(f"\n--- Message {i} (role: {msg['role']}) ---")
    if isinstance(msg.get('content'), list):
        print(f"Content is a list with {len(msg['content'])} item(s)")
        for j, block in enumerate(msg['content']):
            print(f"  Block {j}: type={block.get('type')}, has cache_control={('cache_control' in block)}")
            if 'cache_control' in block:
                print(f"    cache_control: {block['cache_control']}")
    else:
        content_preview = str(msg.get('content', ''))[:100]
        print(f"Content is string: {content_preview}...")